## 🔍 하이브리드 검색(Hybrid Retrieval) 실험실

이 노트북은 **HybridRetriever**의 성능을 테스트하고, 검색 알고리즘을 튜닝하기 위한 공간입니다.
벡터 검색(Vector Search)과 키워드 검색(Keyword Search)이 결합된 RRF(Reciprocal Rank Fusion) 방식의 결과를 확인해보세요.

### 🆕 주요 기능
- **단일 검색 테스트**: 특정 질문에 대한 검색 결과 및 점수 확인
- **성능 평가 (Evaluation)**: Hit@K, MRR 등 정량적 지표 측정

In [1]:
import os
import sys
from dotenv import load_dotenv

# 1. 프로젝트 루트 경로 설정
current_path = os.getcwd()
project_root = current_path

while not os.path.exists(os.path.join(project_root, 'data')) and project_root != '/':
    project_root = os.path.dirname(project_root)

if project_root == '/':
    project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
    
sys.path.append(project_root)
load_dotenv(os.path.join(project_root, ".env"))

print(f"📂 Project Root: {project_root}")

# 모듈 임포트
from src.retrieval.hybrid_search import HybridRetriever
from src.core.config import ZipsaConfig

📂 Project Root: /Users/leemdo/Workspaces/SKN22-3rd-3Team


## ⚙️ 검색 엔진 초기화 (Initializer)
검색할 **버전(Version)**과 **컬렉션(Collection)**을 선택하세요.
- **v2**: Pro 버전 (전문가 필터, 멀티 라벨)
- **v3**: 최신 Clean 버전 (3-Stage Pipeline, Pickle 임베딩)
- **컬렉션**: `care_guides` (가이드), `breeds` (품종)

In [2]:
# ✅ 설정 변경
VERSION = "v3"  # v2, v3
COLLECTION = "care_guides" # care_guides, breeds

print(f"🚀 Initializing HybridRetriever ({VERSION}, {COLLECTION})...")
retriever = HybridRetriever(version=VERSION, collection_name=COLLECTION)

🚀 Initializing HybridRetriever (v3, care_guides)...


## 🧪 1. 단일 검색 테스트 (Ad-hoc Test)
원하는 질문을 입력하여 검색 결과를 확인합니다.
**Specialist** 필터링을 적용하면 특정 전문가의 필터링된 문서만 검색할 수 있습니다.

In [3]:
# ✅ 검색 쿼리 설정
QUERY = "에게안" 
# QUERY = "고양이가 밥을 안 먹어"

# ✅ 전문가 필터 (None이면 전체 검색)
# Options: "Matchmaker", "Physician", "Peacekeeper", "Liaison"
SPECIALIST = None 
# SPECIALIST = "Physician" 

LIMIT = 3

# 검색 실행
results = await retriever.search(query=QUERY, specialist=SPECIALIST, limit=LIMIT)

print(f"🔍 Query: '{QUERY}' (Filter: {SPECIALIST})")
print(f"📊 Found {len(results)} results:\n")

for i, doc in enumerate(results, 1):
    score = doc.get('final_score', 0.0)
    title = doc.get('title_refined', doc.get('title', 'No Title'))
    categories = doc.get('categories', [])
    specs = doc.get('specialists', [])
    
    # 텍스트 일부만 출력
    text_preview = doc.get('text', doc.get('content', ''))[:100].replace("\n", " ") + "..."
    
    print(f"{i}. [{score:.4f}] {title}")
    print(f"   🏷️  Tags: {categories} | 👤 {specs}")
    print(f"   📄 Preview: {text_preview}")
    print("-" * 60)

🔍 [RETRIEVER]: Searching for '에게안' (Specialist: None)...


Quantization is not supported for ArchType::neon. Fall back to non-quantized model.


📚 Loading User Dictionary from: /Users/leemdo/Workspaces/SKN22-3rd-3Team/data/v3/domain_dictionary.txt
🛑 Loaded 28 stopwords from ../core/stopwords.txt
🔄 Loaded 46 flattened synonym mappings from ../core/synonyms.json
✅ [RETRIEVER]: Found 7 results.
🔍 Query: '에게안' (Filter: None)
📊 Found 3 results:

1. [0.0167] 고양이 안기 방법과 주의사항
   🏷️  Tags: ['Care'] | 👤 ['Liaison']
   📄 Preview: 고양이 안기, 좀처럼 쉬운 일은 아니다. 잘 안기는 고양이도 있지만, 안으려할 때마다 몸부림을 치는 고양이도 있다. 하지만 이동장 안에 넣을 때나 손톱을 깎일 때. 고양이를 안아야...
------------------------------------------------------------
2. [0.0167] 에게안 고양이 (고양이 품종)
   🏷️  Tags: ['Breeds'] | 👤 ['Matchmaker']
   📄 Preview: 품종: 에게안 고양이 (Aegean) [개요] 에게안 고양이는 그리스의 키클라데스 제도에서 자연적으로 발생한 고양이로, 사람들과 잘 어울리며 가족과 아이들에게 적합한 성격을 가지고...
------------------------------------------------------------
3. [0.0164] 고양이 높은 곳에서 뛰어내려도 괜찮을까?
   🏷️  Tags: ['Health', 'Behavior'] | 👤 ['Physician']
   📄 Preview: 창문을 여는 화창한 봄이 오면 고양이가 창문에서 뛰어내리는 일이 심심치 않게 일어나곤 합니다. 고양이 높은 곳에서 뛰어내려도 정말 괜찮을까요? 고양이는 높은 곳에서 떨어져도 네 발.

## 📊 2. 성능 평가 (Evaluation)

`data/v3/golden_dataset.json` 파일을 로드하여 **Hit@K**와 **MRR**을 측정합니다.
전체 1,000+개의 데이터셋을 모두 테스트하면 시간이 오래 걸릴 수 있으므로 `SAMPLE_SIZE`를 조절하세요.

In [4]:
import json
import random

# ✅ 데이터셋 로드
dataset_path = os.path.join(project_root, "data/v3/golden_dataset.json")

if os.path.exists(dataset_path):
    with open(dataset_path, "r", encoding="utf-8") as f:
        FULL_DATASET = json.load(f)
    print(f"📚 Loaded {len(FULL_DATASET)} test cases from {dataset_path}")
else:
    print(f"⚠️ Dataset not found at {dataset_path}. Using mock data.")
    FULL_DATASET = [] # Mock data if needed

# ✅ 샘플링 (전체를 다 돌리려면 len(FULL_DATASET)으로 설정)
SAMPLE_SIZE = 50  # 테스트할 개수
TEST_DATASET = random.sample(FULL_DATASET, min(SAMPLE_SIZE, len(FULL_DATASET)))

print(f"🧪 Selected {len(TEST_DATASET)} samples for evaluation.")

# 메트릭 계산 함수
def calculate_metrics(results, expected_keyword):
    """
    단일 쿼리 결과에 대한 Hit 여부와 Rank를 반환
    """
    for rank, doc in enumerate(results, 1):
        # 제목이나 본문에 정답 키워드가 포함되어 있는지 확인
        # V3 필드 대응 (title_refined, text) + V2 Fallback
        title = doc.get('title_refined', doc.get('title', ''))
        text = doc.get('text', doc.get('content', ''))
        summary = doc.get('summary', '')
        
        content = (title + " " + text + " " + summary).lower()
        
        if expected_keyword.lower() in content:
            return 1, 1/rank  # Hit, Reciprocal Rank
    return 0, 0

📚 Loaded 3354 test cases from /Users/leemdo/Workspaces/SKN22-3rd-3Team/data/v3/golden_dataset.json
🧪 Selected 50 samples for evaluation.


In [6]:
import pandas as pd
import time

async def run_evaluation(test_set, k=5):
    print(f"🚀 Starting Evaluation on {len(test_set)} queries (Limit K={k})...\n")
    
    total_hits = 0
    total_mrr = 0
    results_log = []
    
    for i, case in enumerate(test_set, 1):
        query = case.get("query")
        # 'expected_keyword' key used in json, fallback to 'expected' just in case
        expected = case.get("expected_keyword", case.get("expected")) 
        specialist = case.get("specialist")
        
        # 진행상황 표시 (10개 단위)
        if i % 10 == 0: print(f"...Processing {i}/{len(test_set)}")
        
        # 검색 실행
        start_time = time.time()
        # Specialist None 처리 (데이터셋에 None으로 저장된 경우)
        if specialist == "None": specialist = None
            
        docs = await retriever.search(query=query, specialist=specialist, limit=k)
        elapsed = time.time() - start_time
        
        # 메트릭 계산
        is_hit, rr = calculate_metrics(docs, expected)
        total_hits += is_hit
        total_mrr += rr
        
        # 로그 저장
        top_doc_title = "-"
        if docs:
            top_doc_title = docs[0].get('title_refined', docs[0].get('title', 'No Result'))
            
        results_log.append({
            "Query": query,
            "Target": expected,
            "Filter": specialist,
            "Hit": "✅" if is_hit else "❌",
            "Rank": int(1/rr) if rr > 0 else "-",
            "Latency": f"{elapsed:.3f}s",
            "Top1_Title": top_doc_title
        })
    
    # 결과 출력
    df = pd.DataFrame(results_log)
    
    hit_rate = (total_hits / len(test_set)) * 100
    mrr_score = total_mrr / len(test_set)
    
    print("="*60)
    print(f"🏆 Evaluation Results (K={k})")
    print("="*60)
    print(f"🎯 Hit@{k}   : {hit_rate:.1f}%")
    print(f"🏅 MRR      : {mrr_score:.4f}")
    print("="*60)
    
    return df

# 평가 실행
df_results = await run_evaluation(TEST_DATASET, k=3)
df_results.head(20)  # 상위 20개 결과 확인

🚀 Starting Evaluation on 50 queries (Limit K=3)...

🔍 [RETRIEVER]: Searching for '고양이 화장실은 얼마나 자주 청소해야 하나요?' (Specialist: General)...
✅ [RETRIEVER]: Found 10 results.
🔍 [RETRIEVER]: Searching for '고양이와의 놀이 방법은?' (Specialist: Peacekeeper)...
✅ [RETRIEVER]: Found 9 results.
🔍 [RETRIEVER]: Searching for '고양이의 스트레스 해소 방법은?' (Specialist: Peacekeeper)...
✅ [RETRIEVER]: Found 11 results.
🔍 [RETRIEVER]: Searching for '고양이에게 안전한 식물은 무엇인가요?' (Specialist: Peacekeeper)...
✅ [RETRIEVER]: Found 9 results.
🔍 [RETRIEVER]: Searching for '추천 유산균 제품은?' (Specialist: Physician)...
✅ [RETRIEVER]: Found 10 results.
🔍 [RETRIEVER]: Searching for '아기 고양이와 성묘의 차이는 무엇인가요?' (Specialist: Physician)...
✅ [RETRIEVER]: Found 10 results.
🔍 [RETRIEVER]: Searching for '어떤 제품이 좋을까?' (Specialist: Peacekeeper)...
✅ [RETRIEVER]: Found 11 results.
🔍 [RETRIEVER]: Searching for '정기 검진은 얼마나 자주 받아야 하나요?' (Specialist: Physician)...
✅ [RETRIEVER]: Found 11 results.
🔍 [RETRIEVER]: Searching for '고양이와의 신뢰 관계는 어떻게 형성되나요?' (Specialist:

,Query,Target,Filter,Hit,Rank,Latency,Top1_Title
0,고양이 화장실은 얼마나 자주 청소해야 하나요?,고양이 화장실 청소 방법과 주기,General,✅,1,0.813s,고양이 화장실 청소 방법과 주기
1,고양이와의 놀이 방법은?,고양이가 일부러 물건을 떨어뜨리는 이유,Peacekeeper,❌,-,0.390s,고양이 호기심을 만족시키는 놀이 방법
2,고양이의 스트레스 해소 방법은?,고양이의 행동 신호 이해하기,Peacekeeper,❌,-,0.371s,고양이 어항 쳐다보는 이유: 스트레스 해소 효과
3,고양이에게 안전한 식물은 무엇인가요?,고양이와 식물 안전하게 함께하기,Peacekeeper,✅,1,0.698s,고양이와 식물 안전하게 함께하기
4,추천 유산균 제품은?,고양이 유산균과 변비 해결법,Physician,✅,1,0.391s,고양이 유산균과 변비 해결법
5,아기 고양이와 성묘의 차이는 무엇인가요?,아기 고양이 성장과 성묘의 차이점 5가지,Physician,✅,1,0.397s,아기 고양이 성장과 성묘의 차이점 5가지
6,어떤 제품이 좋을까?,고양이 소파 스크래쳐 추천 8가지,Peacekeeper,❌,-,0.341s,펫도어 선택 가이드 및 추천 10가지
7,정기 검진은 얼마나 자주 받아야 하나요?,노령묘 건강 관리 5가지 방법,Physician,❌,-,0.458s,고양이 건강검진: 필수 가이드
8,고양이와의 신뢰 관계는 어떻게 형성되나요?,고양이 잠자리 장소의 비밀,Peacekeeper,❌,-,0.524s,고양이 역사와 인간과의 관계
9,구내염 치료 방법은?,고양이 구내염 증상 및 진단,Physician,✅,3,0.620s,고양이 구내염 완벽 가이드
